In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import pathlib
import os
import zipfile
from multiprocessing import Pool

%matplotlib inline
%load_ext autoreload
%autoreload 2

# Explore the Different CSVs

In [2]:
# to check CPU count
import multiprocessing
print('CPUs avail:', multiprocessing.cpu_count()) # or os.cpu_count()

# memory available
# https://stackoverflow.com/a/48140392/9214620
!cat /proc/meminfo | grep Mem

CPUs avail: 32
MemTotal:       131624740 kB
MemFree:        51615444 kB
MemAvailable:   91471860 kB


In [5]:
root_dir = Path.cwd().parent
folder_raw_data = root_dir / 'data/raw'
folder_processed_data = root_dir / 'data/processed'
folder_external_data = root_dir / 'data/external'

In [6]:
def extract_zip(folder_with_zips, year_list_to_extract=None):
    """function to extract the csv's from the zip files"""
    # get all the zip file names in folder_with_zips
    files = os.listdir(folder_with_zips)
    
    # create list of all the files
    file_list = [folder_raw_data / filename for filename in files if filename.endswith('.zip')]
    for zip_file in file_list:
        year = int(str(zip_file).split('/')[-1][4:8])
        if year_list_to_extract == None:
            with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                print('extracting:', str(zip_file).split('/')[-1])
                zip_ref.extractall(folder_with_zips)
        elif year in year_list_to_extract:
            with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                print('extracting:', str(zip_file).split('/')[-1])
                zip_ref.extractall(folder_with_zips)
        else:
            pass                      

In [7]:
extract_the_csvs = False

# extract the csv's if the above is True
# if extract_the_csvs:
#     extract_zip(folder_raw_data, year_list_to_extract=list(range(1968,1980)))
    
# # rename select CSVs that have a different naming format
# # 2018 file uses different naming convention
# file_2018 = folder_raw_data / 'natl2018us.csv'
# if file_2018.is_file():
#     os.rename(file_2018, folder_raw_data / 'natl2018.csv')
    
# # 2019 file uses different naming convention
# file_2019 = folder_raw_data / 'birth_2019_nber_us_v2.csv'
# if file_2019.is_file():
#     os.rename(file_2019, folder_raw_data / 'natl2019.csv')

In [10]:
# get a list of file names
files = os.listdir(folder_raw_data)
file_list = [folder_raw_data / filename for filename in files if filename.endswith('.csv')]
# file_list

The CSV is very large, therefore we will only load certain columns. The columns names are listed in the description pdf. These are the ones we will use for 2003-2004:
* **dob_yy**: date of birth year
* **dob_mm**: date of birth month
* **dob_wk**: date of birth weekday
* **mrstate**: mother's resident state
* **mrecntyfips**: mother's resident county FIPS code
* **mrcityfips**: mother's place of residence (city) FIPS code

Geographic data is not available from 2005 onwards. Therefore, only these columns will be used.
* **dob_yy**: date of birth year
* **dob_mm**: date of birth month
* **dob_wk**: date of birth weekday


Depending on the year, there are different naming conventions. Here's the ones we'll use for 1989 through 2002 (see [column description for 1989 on NBER](https://data.nber.org/natality/1989/desc/natl1989/desc.txt)):

* **biryr**: date of birth year (can also use 'datayear')
* **birmon**: date of birth month
* **weekday**: date of birth weekday
* **stresfip**: mother's resident state FIPS code
* **cntyrfip**: mother's resident county FIPS code
* **cityres**: mother's place of residence (city) - unsure if fips code


Here's the ones we'll use for 1982 through 1988 (see [column description on NBER](https://data.nber.org/natality/1988/desc/natl1988/desc.txt)):

* **datayear**: date of birth year
* **birmon**: date of birth month
* **birday**: birth date - day (like the 15th of June) (1968-1988 only)
* ~~**weekday**: date of birth weekday~~ (does not exist in this date range)
* **stresfip**: mother's resident state FIPS code
* **cntyrfip**: mother's resident county FIPS code
* **cityres**: mother's place of residence (city) - unsure if fips code

In [11]:
# columns for 2003 through 2004
col_load_1 = ['dob_yy','dob_mm','dob_wk','mrstate','mrcntyfips','mrcityfips',]
col_1_dtype = [int, int, int, str, int, int]

# columns for 1989 through 2002
col_load_2 =['biryr', 'birmon', 'weekday', 'stresfip', 'cntyrfip', 'cityres']

# columns for 2005+
col_load_3 =['dob_yy','dob_mm','dob_wk']

# columns for 1982 through 1988
col_load_4 =['datayear', 'birmon','birday','stresfip', 'cntyrfip', 'cityres']
rename_col4 = ['dob_yy','dob_mm','dob_day','mrstate','mrcntyfips','mrcityfips',]

# create dictionary to rename older csvs
col_rename_dict = dict(zip(col_load_2, col_load_1))
col_rename_dict4 = dict(zip(col_load_4, rename_col4))
col_rename_dict

{'biryr': 'dob_yy',
 'birmon': 'dob_mm',
 'weekday': 'dob_wk',
 'stresfip': 'mrstate',
 'cntyrfip': 'mrcntyfips',
 'cityres': 'mrcityfips'}

Load "older" csv (from 1991-2002)

In [12]:
year = 1993
df = pd.read_csv(folder_raw_data / f'natl{str(year)}.csv', nrows=1, usecols=col_load_2, dtype=str).rename(columns=col_rename_dict)
df = df.rename(columns={'mrstate':'mrstatefips'})
df.head()

,mrcityfips,mrstatefips,mrcntyfips,dob_mm,dob_yy,dob_wk
0,999,01,01999,1,1993,2


In [14]:
# load 'all-geocodes-v2017.xlsx'
# df_fips = pd.read_csv('./other_data/all-geocodes-v2017.csv', 
#                         dtype={'State Code (FIPS)': int, 
#                                'County Code (FIPS)': int, 
#                                'County Subdivision Code (FIPS)': int, 
#                                'Place Code (FIPS)': int, 
#                                'Consolidtated City Code (FIPS)': int})

df_fips = pd.read_csv(folder_external_data / 'all-geocodes-v2017.csv', dtype=str)

df_fips.head()

,Summary Level,State Code (FIPS),County Code (FIPS),County Subdivision Code (FIPS),Place Code (FIPS),Consolidtated City Code (FIPS),Area Name (including legal/statistical area description)
0,010,00,000,00000,00000,00000,United States
1,040,01,000,00000,00000,00000,Alabama
2,050,01,001,00000,00000,00000,Autauga County
3,050,01,003,00000,00000,00000,Baldwin County
4,050,01,005,00000,00000,00000,Barbour County


In [15]:
# get the fips codes for the states only
# df_state_fips = df_fips[(df_fips['State Code (FIPS)']>0) & 
#                         (df_fips['County Code (FIPS)']==0) & 
#                         (df_fips['County Subdivision Code (FIPS)']==0) & 
#                         (df_fips['Place Code (FIPS)']==0) & 
#                         (df_fips['Consolidtated City Code (FIPS)']==0)
#                        ][['State Code (FIPS)','Area Name (including legal/statistical area description)']]


df_state_fips = df_fips[(df_fips['County Code (FIPS)']=='000') & 
                        (df_fips['County Subdivision Code (FIPS)']=='00000') & 
                        (df_fips['Place Code (FIPS)']=='00000') & 
                        (df_fips['Consolidtated City Code (FIPS)']=='00000')
                       ][['State Code (FIPS)','Area Name (including legal/statistical area description)']]

# rename columns in df
df_state_fips.columns = ['state_fips', 'state_name_mr']
df_state_fips.head()

,state_fips,state_name_mr
0,00,United States
1,01,Alabama
530,02,Alaska
708,04,Arizona
815,05,Arkansas


In [16]:
df = pd.merge(df, df_state_fips, left_on='mrstatefips',
              right_on='state_fips', how='inner', copy=False).drop(['state_fips'], axis=1).drop(['mrstatefips'], axis=1)
df.head()

,mrcityfips,mrcntyfips,dob_mm,dob_yy,dob_wk,state_name_mr
0,999,01999,1,1993,2,Alabama


In [17]:
# change the dtype for the numerical columns
df = df.astype({'dob_mm':int, 'dob_wk':int, 'dob_yy':int})
df.head()

,mrcityfips,mrcntyfips,dob_mm,dob_yy,dob_wk,state_name_mr
0,999,01999,1,1993,2,Alabama


Load 1982-1988.

In [18]:
year = 1982
df = pd.read_csv(folder_raw_data / f'natl{str(year)}.csv', nrows=1, usecols=col_load_4, dtype=str).rename(columns=col_rename_dict4)
df = df.rename(columns={'mrstate':'mrstatefips'})
df.head()

,dob_yy,mrcityfips,dob_mm,dob_day,mrstatefips,mrcntyfips
0,2,999,1,2,01,01001


In [19]:
df = pd.merge(df, df_state_fips, left_on='mrstatefips',
              right_on='state_fips', how='inner', copy=False).drop(['state_fips'], axis=1).drop(['mrstatefips'], axis=1)
df.head()

,dob_yy,mrcityfips,dob_mm,dob_day,mrcntyfips,state_name_mr
0,2,999,1,2,01001,Alabama


Load 2005+.

In [20]:
year = 2019

# columns for 2005+
col_load_3 =['dob_yy','dob_mm','dob_wk',]

df = pd.read_csv(folder_raw_data / f'natl{str(year)}.csv', nrows=1, usecols=col_load_3, dtype=str)
df

,dob_yy,dob_mm,dob_wk
0,2019,01,3


Load 2003-2004 csv.

In [21]:
year = 2003
df = pd.read_csv(folder_raw_data / f'natl{str(year)}.csv', nrows=1, usecols=col_load_1, dtype=str)
df

,dob_yy,dob_mm,dob_wk,mrstate,mrcntyfips,mrcityfips
0,2003,1,4,AK,999,99999


In [22]:
# load 'state_abbreviations.csv'
df_abbr = pd.read_csv(folder_external_data / 'state_abbreviations.csv',header=None, names=['state','abbr'])
df_abbr.head()

,state,abbr
0,Alabama,AL
1,Alaska,AK
2,American Samoa,AS
3,Arizona,AZ
4,Arkansas,AR


In [23]:
# use pandas merge. Also, dorp the 'abbr' columns right away
df = pd.merge(df, df_abbr, left_on='mrstate',right_on='abbr', how='inner', copy=False).drop(['abbr'], axis=1).drop(['mrstate'], axis=1)
df = df.rename(columns={'state':'state_name_mr'})
df = df.astype({'dob_mm':int, 'dob_wk':int, 'dob_yy':int})
df.head()

,dob_yy,dob_mm,dob_wk,mrcntyfips,mrcityfips,state_name_mr
0,2003,1,4,999,99999,Alaska


In [24]:
# add state fips code
df = pd.merge(df, df_state_fips, left_on='state_name_mr',
              right_on='state_name_mr', how='inner', copy=False)
df = df.rename(columns={'state_fips':'mrstatefips'})
df.head()

,dob_yy,dob_mm,dob_wk,mrcntyfips,mrcityfips,state_name_mr,mrstatefips
0,2003,1,4,999,99999,Alaska,02


Practice grouping a table together by birth year/month/week-day.

In [25]:
year = 2005

# columns for 2005+
col_load_3 =['dob_yy','dob_mm','dob_wk',]

df = pd.read_csv(folder_raw_data / f'natl{str(year)}.csv', nrows=100, usecols=col_load_3, dtype=int)
df.head()

,dob_yy,dob_mm,dob_wk
0,2005,1,3
1,2005,1,4
2,2005,1,7
3,2005,1,7
4,2005,1,7


In [26]:
df.shape

(100, 3)

In [27]:
df['births'] = np.ones(df.shape[0])
df1 = df.groupby(['dob_yy', 'dob_mm', 'dob_wk'], as_index=False).count().sort_values(by=['dob_yy','dob_mm','dob_wk'])
df1[:10]

,dob_yy,dob_mm,dob_wk,births
0,2005,1,1,11
1,2005,1,2,16
2,2005,1,3,17
3,2005,1,4,15
4,2005,1,5,14
5,2005,1,6,7
6,2005,1,7,20


The final dataframe is much smaller - only 84 rows.

In [28]:
df1.shape

(7, 4)

# Putting it all Together
Create a function that loads a csv and returns a formatted data frame.

In [31]:
def df_from_csv_with_geo(file_path, nrows=None):
    """Extract useful columns from birth record csv
    Takes a csv path. CSV must be before 2005 to include geo data.   
    """
    
    # get year of CSV
    year = int(str(file_path).split('/')[-1][4:8])
    
    if year > 2004 or year < 1982:
        pass
    
    else:
        
        # load FIPS code data
        df_fips = pd.read_csv(folder_external_data / 'all-geocodes-v2017.csv', dtype=str)


        # get the fips codes for the states only
        df_fips = df_fips[(df_fips['County Code (FIPS)']=='000') & 
                                (df_fips['County Subdivision Code (FIPS)']=='00000') & 
                                (df_fips['Place Code (FIPS)']=='00000') & 
                                (df_fips['Consolidtated City Code (FIPS)']=='00000')
                               ][['State Code (FIPS)','Area Name (including legal/statistical area description)']]

        # rename columns in df
        df_fips.columns = ['state_fips', 'state_name_mr']
        
        # require differnt column names depending on year
        # columns for 2003+
        col_load_1 = ['dob_yy','dob_mm','dob_wk','mrstate','mrcntyfips','mrcityfips',]

        # columns for 1989-2002
        col_load_2 =['biryr', 'birmon', 'weekday', 'stresfip', 'cntyrfip', 'cityres']
        rename_col2 = ['dob_yy','dob_mm','dob_wk','mrstate','mrcntyfips','mrcityfips',]
        
        # columns for 1982 through 1988
        col_load_3 =['datayear', 'birmon','birday','stresfip', 'cntyrfip', 'cityres']
        rename_col3 = ['dob_yy','dob_mm','dob_day','mrstate','mrcntyfips','mrcityfips',]

        # create dictionary to rename older csvs
        col_rename_dict2 = dict(zip(col_load_2, rename_col2))
        col_rename_dict3 = dict(zip(col_load_3, rename_col3))

        # if the CSVs are of newer format
        if year >= 2003:  
            # load abbreviation csv so we can rename AK to Alaska, etc.
            df_abbr = pd.read_csv(folder_external_data / 'state_abbreviations.csv',header=None, names=['state','abbr'])

            # load only select columns, and set dtype for columns
            df = pd.read_csv(file_path, nrows=nrows, usecols=col_load_1, dtype=str)

            # get the full state name and append them onto the df
            df = pd.merge(df, df_abbr, left_on='mrstate',right_on='abbr', how='inner', copy=False).drop(['abbr'], axis=1).drop(['mrstate'], axis=1)
            df = df.rename(columns={'state':'state_name_mr'})
            
            # get state FIPS code and append
            df = pd.merge(df, df_fips, left_on='state_name_mr',right_on='state_name_mr', how='inner', copy=False)
            df = df.rename(columns={'state_fips':'mrstatefips'})

            # drop any rows with NaN's
            df = df.dropna()

        # if the CSVs are of older format
        else:

            
            if year >= 1989:
                col_load, col_rename_dict = col_load_2, col_rename_dict2
            else:
                col_load, col_rename_dict = col_load_3, col_rename_dict3

            # load only select columns from the birth CSV
            df = pd.read_csv(file_path, nrows=nrows, usecols=col_load, dtype=str).rename(columns=col_rename_dict)

            # rename 'mrstate' column 
            df = df.rename(columns={'mrstate':'mrstatefips'})

            # merge the df_stat_fips to get the full state name
            df = pd.merge(df, df_fips, left_on='mrstatefips',
                  right_on='state_fips', how='inner', copy=False).drop(['state_fips'], axis=1)
            
            # years before 1989 only show a single digit (i.e. 2 for 1982)
            if year < 1989:
                df = df.drop(columns=['dob_yy'])
                df['dob_yy'] = np.array([year]*df.shape[0])

            # drop any rows with NaN's
            df = df.dropna()
    

        # return the dataframe, and order the columns in a fixed manner
        return df[['dob_yy', 'dob_mm',
                   'mrcntyfips', 'mrcityfips', 'state_name_mr', 'mrstatefips']].astype({'dob_mm':int, 'dob_yy':int})


In [32]:
%%time
# test out the loader
year = 1986

df = df_from_csv_with_geo(folder_raw_data / f'natl{str(year)}.csv', nrows=3)
df.head()

CPU times: user 106 ms, sys: 15.2 ms, total: 121 ms
Wall time: 187 ms


,dob_yy,dob_mm,mrcntyfips,mrcityfips,state_name_mr,mrstatefips
0,1986,1,01003,999,Alabama,01
1,1986,1,01003,999,Alabama,01
2,1986,1,01003,999,Alabama,01


Create a function to load the CSVs using multiple processors. This should speed things up, but you'll have to make sure you have enough RAM. Otherwise, just use a simple sequential for loop.

In [33]:
%%time
def main(folder_raw_data):

    # get a list of file names
    files = os.listdir(folder_raw_data)
    file_list = [folder_raw_data / filename for filename in files if filename.endswith('.csv')]

    # set up your pool
    with Pool(processes=20) as pool: # or whatever your hardware can support

        # have your pool map the file names to dataframes
        df_list = pool.map(df_from_csv_with_geo, file_list)

        # reduce the list of dataframes to a single dataframe
        combined_df = pd.concat(df_list, ignore_index=True)
        
        return combined_df
        

if __name__ == '__main__':
    df = main(folder_raw_data)
    
print('Final df shape:',df.shape)

Final df shape: (89598797, 6)
CPU times: user 19.7 s, sys: 17.2 s, total: 36.9 s
Wall time: 1min 22s


In [34]:
df.head()

,dob_yy,dob_mm,mrcntyfips,mrcityfips,state_name_mr,mrstatefips
0,1989,1,01999,999,Alabama,01
1,1989,1,01999,999,Alabama,01
2,1989,1,01999,999,Alabama,01
3,1989,1,01999,999,Alabama,01
4,1989,1,01999,999,Alabama,01


In [35]:
# create a birth count for each unique geo and date
# this should reduce the size of the df significantly
df['births'] = np.ones(df.shape[0])
df = df.groupby(list(df.columns)[:-1], as_index=False).count().sort_values(by=['dob_yy','dob_mm'])
df.head()

,dob_yy,dob_mm,mrcntyfips,mrcityfips,state_name_mr,mrstatefips,births
0,1982,1,00000,ZZZ,United States,00,483
1,1982,1,01001,999,Alabama,01,44
2,1982,1,01003,999,Alabama,01,90
3,1982,1,01005,999,Alabama,01,33
4,1982,1,01007,999,Alabama,01,22


In [36]:
df.shape

(409414, 7)

In [37]:
# save the dataframe to a zipped csv
# https://stackoverflow.com/a/46073397/9214620
save_csv = True
if save_csv:
    df.to_csv(folder_processed_data / 'birth_geo.csv.gz', compression='gzip', index=False)

Now create a function to only get the info for birth dates.

In [27]:
def df_from_csv_no_geo(file_path, nrows=None):
    """Extract useful columns from birth record csv
    Takes a csv path. Produces a dataframe without geo data.
    Good for all years of data collection.
    """
    
    # get year of CSV
    year = int(str(file_path).split('/')[-1][4:8])
       
    # require differnt column names depending on year
    # columns for 2003+
    col_load_1 = ['dob_yy','dob_mm']

    # columns for 1989-2002
    col_load_2 =['biryr', 'birmon']
    rename_col2 = ['dob_yy','dob_mm']

    # columns for 1982 through 1988
    col_load_3 =['datayear', 'birmon']
    rename_col3 = ['dob_yy','dob_mm']

    # create dictionary to rename older csvs
    col_rename_dict2 = dict(zip(col_load_2, rename_col2))
    col_rename_dict3 = dict(zip(col_load_3, rename_col3))
    
    # create dictionary to rename older CSVs from 1991 to 2002
    col_rename_dict = dict(zip(col_load_2, col_load_1))

    # if the CSVs are of newer format
    if year >= 2003:  

        # load only select columns, and set dtype for columns
        df = pd.read_csv(file_path, nrows=nrows, usecols=col_load_1, dtype=str)
        
        # drop any rows with NaN's
        df = df.dropna()
        
        df['births'] = np.ones(df.shape[0])
        df = df.groupby(['dob_yy', 'dob_mm'], as_index=False).count().sort_values(by=['dob_yy','dob_mm'])

    elif year > 1988 and year < 2004:
        
        # load only select columns from the birth CSV
        df = pd.read_csv(file_path, nrows=nrows, usecols=col_load_2, dtype=str).rename(columns=col_rename_dict2)

        # drop any rows with NaN's
        df = df.dropna()
        
        df['births'] = np.ones(df.shape[0])
        df = df.groupby(['dob_yy', 'dob_mm'], as_index=False).count().sort_values(by=['dob_yy','dob_mm'])
        

    # if the CSVs are of older format
    else:

        # load only select columns from the birth CSV
        df = pd.read_csv(file_path, nrows=nrows, usecols=col_load_3, dtype=str).rename(columns=col_rename_dict3)
        
        # years before 1989 only show a single digit (i.e. 2 for 1982)
        df = df.drop(columns=['dob_yy'])
        df['dob_yy'] = np.array([year]*df.shape[0])

        # drop any rows with NaN's
        df = df.dropna()
        
        df['births'] = np.ones(df.shape[0])
        df = df.groupby(['dob_yy', 'dob_mm'], as_index=False).count().sort_values(by=['dob_yy','dob_mm'])

    # return the dataframe, and order the columns in a fixed manner
    return df[['dob_yy', 'dob_mm', 'births']].astype({'dob_mm':int, 'dob_yy':int, 'births':int})


In [41]:
%%time
# test out the loader
year = 1968

df = df_from_csv_no_geo(folder_raw_data / f'natl{str(year)}.csv', nrows=6)
df.head()

CPU times: user 22.1 ms, sys: 2.78 ms, total: 24.9 ms
Wall time: 24.5 ms


,dob_yy,dob_mm,births
0,1968,1,6


In [42]:
%%time
def main(folder_raw_data):

    # get a list of file names
    files = os.listdir(folder_raw_data)
    file_list = [folder_raw_data / filename for filename in files if filename.endswith('.csv')]

    # set up your pool
    with Pool(processes=25) as pool: # or whatever your hardware can support

        # have your pool map the file names to dataframes
        df_list = pool.map(df_from_csv_no_geo, file_list)

        # reduce the list of dataframes to a single dataframe
        combined_df = pd.concat(df_list, ignore_index=True)
        
        return combined_df
        

if __name__ == '__main__':
    df = main(folder_raw_data)
    
print('Final df shape:',df.shape)

Final df shape: (624, 3)
CPU times: user 203 ms, sys: 176 ms, total: 380 ms
Wall time: 1min 30s


In [44]:
df[df['dob_yy']==1968].head()

,dob_yy,dob_mm,births
108,1968,1,138702
109,1968,10,152144
110,1968,11,145792
111,1968,12,152383
112,1968,2,133041


In [45]:
# save the dataframe to a csv
save_csv = True
if save_csv:
    df.to_csv(folder_processed_data / 'birth_no_geo.csv', index=False)

In [46]:
df.shape

(624, 3)